In [1]:
#Importing the packages
import cv2
import dlib
import numpy as np
import tensorflow as tf
import time
from playsound import playsound
import threading

In [2]:
# Load the pre-trained eye model
eye_model = tf.keras.models.load_model('eye_model.h5')

# Haarcascades for face and eyes
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')

# Dlib's face detector and landmark predictor
hog_face_detector = dlib.get_frontal_face_detector()
dlib_facelandmark = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

In [3]:
# Function to predict eye state
def predict_eye_state(eye_image):
    try:
        eye_image = cv2.resize(eye_image, (64, 64))
        eye_image = cv2.cvtColor(eye_image, cv2.COLOR_GRAY2RGB)
        eye_image = eye_image / 255.0
        eye_image = np.expand_dims(eye_image, axis=0)
        prediction = eye_model.predict(eye_image, verbose=0)
        return 'Closed' if prediction[0] < 0.5 else 'Open'
    except:
        return 'Unknown'

# Mouth aspect ratio for yawning
def mouth_aspect_ratio(landmarks):
    top_lip = landmarks.part(62).y
    bottom_lip = landmarks.part(66).y
    left_corner = landmarks.part(48).x
    right_corner = landmarks.part(54).x
    return (bottom_lip - top_lip) / (right_corner - left_corner)

EYE_CLOSED_THRESHOLD_SEC = 5.0
MAR_THRESHOLD = 0.4

eye_closed_start_time = None
is_drowsy = False
alarm_on = False

def play_alarm():
    global alarm_on
    if not alarm_on:
        alarm_on = True
        playsound("alarm.wav")  # Replace with your sound file
        alarm_on = False

In [5]:
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    current_time = time.time()

    # Dlib for landmarks (mouth)
    faces_dlib = hog_face_detector(gray)
    yawn_detected = False
    for face in faces_dlib:
        landmarks = dlib_facelandmark(gray, face)
        mar = mouth_aspect_ratio(landmarks)
        
        if mar > MAR_THRESHOLD:
            yawn_detected = True
            cv2.putText(frame, "Yawning!", (face.left(), face.top() - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)

    # Haar cascade for eyes
    faces = face_cascade.detectMultiScale(gray, 1.1, 5)
    eyes_closed = False
    for (x, y, w, h) in faces:
        roi_gray = gray[y:y + h, x:x + w]
        roi_color = frame[y:y + h, x:x + w]

        eyes = eye_cascade.detectMultiScale(roi_gray)
        for (ex, ey, ew, eh) in eyes:
            eye_image = roi_gray[ey:ey + eh, ex:ex + ew]
            eye_state = predict_eye_state(eye_image)

            cv2.rectangle(roi_color, (ex, ey), (ex + ew, ey + eh), (0, 255, 0), 2)
            cv2.putText(roi_color, f"{eye_state}", (ex, ey - 5),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

            if eye_state == 'Closed':
                eyes_closed = True

    # Tracking eye closure duration
    eye_closure_duration = 0
    is_drowsy = False  # Reset drowsy flag each frame

    if eyes_closed:
        if eye_closed_start_time is None:
            eye_closed_start_time = current_time
        eye_closure_duration = current_time - eye_closed_start_time
    else:
        eye_closure_duration = 0
        eye_closed_start_time = None

    # Set drowsiness based on condition
    if eye_closure_duration >= EYE_CLOSED_THRESHOLD_SEC or yawn_detected:
        is_drowsy = True

        cv2.putText(frame, "DROWSY!!!", (50, 50), cv2.FONT_HERSHEY_SIMPLEX,
                    1.2, (0, 0, 255), 3)
        if not alarm_on:
            threading.Thread(target=play_alarm).start()
    else:
        cv2.putText(frame, "Awake", (50, 50), cv2.FONT_HERSHEY_SIMPLEX,
                    1.2, (0, 255, 0), 2)
        alarm_on = False

    cv2.imshow("Drowsiness Detection", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


    Error 263 for command:
        close alarm.wav
    The specified device is not open or is not recognized by MCI.
Failed to close the file: alarm.wav
